**IMPORT NECESSARY PACKAGES**

In [ ]:
import pandas as pd

**MOUNTING THE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**READING THE CSV FILE**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/eng_movie_subtitles.csv')

In [ ]:
df.head(5)

,Unnamed: 0,name,file_content
0,0,Criminal Minds S10 E20 A Place At The Table,script info title default file scripttype v400...
1,1,Ghost In The Shell Stand Alone Complex S01 E09...,1 angels and demons were circling over my head...
2,2,Halo S01 E04 Homecoming,script info title default file scripttype v400...
3,3,Make My Day S01 E07 Episode 1 7,support us and become vip member to remove all...
4,4,Australian Survivor S04 E11 Episode 4 11,1 ericsson access services 2 and ben soon turn...


In [ ]:
df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

In [ ]:
df.columns

Index(['index', 'name', 'file_content'], dtype='object')

**CHUNCKING & SENTENCE TRANSFORMERS:**

In order to convert the text data in the subtitle contents into vector, I have used BERT-based transformer since it contains an encoder that can understand the context and do embedding for each word.

Since the dataset is huge, there is a chance of missing few words, so the data is splitted into multiple chunks and the embedding is performed on each chunks.

In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer

def generate_embeddings(texts):
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    embeddings = []
    for text_chunk in texts:
        chunk_embeddings = model.encode(text_chunk)
        embeddings.append(chunk_embeddings)
    return embeddings

def document_chunker(data, chunk_size=500, overlap_size=50):
    chunks = []
    start_idx = 0
    while start_idx < len(data):
        end_idx = min(start_idx + chunk_size, len(data))
        chunk = ' '.join(data[start_idx:end_idx])
        chunks.append(chunk)
        start_idx += chunk_size - overlap_size
    return chunks

In [ ]:
texts = df['file_content'].tolist()

In [ ]:
chunked_texts = document_chunker(texts)

In [ ]:
chunk_embeddings = generate_embeddings(chunked_texts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**STORING THE VECTORS IN CHROMADB**

The embedded vectors are stored in Chroma DB. ChromaDB is a database used for storing vectors.

In [ ]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 40

**CREATING A COLLECTION IN CHROMADB AND ADDING THE EMBEDDED VECTORS INTO IT:**

In [ ]:
import chromadb

In [ ]:
client = chromadb.PersistentClient(path="/content/drive/MyDrive/eng_subtitles")


In [ ]:
collection = client.get_or_create_collection(name="eng_subtitles")

In [ ]:
for index, row in df.iterrows():
    document_id = str(row['index'])  # Use a unique identifier for each document
    document_text = row['file_content']
    # document_embedding = corpus_embeddings[index % len(corpus_embeddings)]  # Assuming embeddings are aligned with the rows

    # ------------------------------------------------------------------------
    chunk_index = index % len(chunk_embeddings)
    document_embedding = chunk_embeddings[chunk_index]
    #--------------------------------------------------------------------------
    metadata = {'movie_name': row['name']}

    # Insert document into ChromaDB collection
    collection.add(ids=document_id, documents=[document_text], embeddings=[document_embedding.tolist()], metadatas=[metadata])


print("Insertion into ChromaDB collection complete")

Insertion into ChromaDB collection complete
